# **Data Preparation and Imports**

In [ ]:
!pip install transformers datasets scikit-learn pandas numpy
!pip install --upgrade transformers

# Standard libraries
import os
import random

# Data manipulation libraries
import numpy as np
import pandas as pd

# Machine learning metrics
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score
)

# PyTorch related
import torch
from torch.optim.swa_utils import AveragedModel
from safetensors.torch import load_file

# Hugging Face transformers
from transformers import (
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    Trainer,
    TrainerCallback,
    TrainingArguments
)
from datasets import Dataset

# Google Colab
from google.colab import drive

# Set Constant Seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

# Mount Drive
drive.mount('/content/drive/')
save_path = '/content/drive/MyDrive/nlu'
os.makedirs(save_path, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

## Data Loading and Distribution Calculation

In [ ]:
# Load data
train_df = pd.read_csv('/content/drive/MyDrive/nlu/train.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/nlu/dev.csv')

# Conver Label Column from String to INT
train_df["label"] = train_df["label"].astype(int)
valid_df["label"] = valid_df["label"].astype(int)

# Show Sample of Train Data
train_df.head()


,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [ ]:
# Calculate Weights for both classes
def calculate_normalized_class_weights(train_data):
    y_train = train_data['label'].values

    # Calculate class distribution
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_percentages = class_counts / total_samples * 100

    # Print distribution statistics
    for class_idx, (count, percentage) in enumerate(zip(class_counts, class_percentages)):
        print(f"Class {class_idx}: {count} samples ({percentage:.2f}%)")

    epsilon = 1e-6
    raw_class_weights = total_samples / (len(class_counts) * (class_counts + epsilon))

    normalized_class_weights = raw_class_weights / np.sum(raw_class_weights)

    class_weights = {
        i: normalized_class_weights[i] for i in range(len(normalized_class_weights))
    }

    return class_weights

class_weights = calculate_normalized_class_weights(train_df)

Class 0: 11784 samples (48.23%)
Class 1: 12648 samples (51.77%)


## Tokenization and Utility Functions for ULMFit Training

In [ ]:
# Initialize Roberta Tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-large") # Using Roberta-Large which has 24 layers
def tokenize(batch):
    return tokenizer(batch["premise"], batch["hypothesis"], truncation=True, padding="max_length", max_length=128)

# Convert Dataframes to Dataset Objects and convert to an appropriate format for huggingface trainer
train_ds = Dataset.from_pandas(train_df).map(tokenize, batched=True).rename_column("label", "labels")
valid_ds = Dataset.from_pandas(valid_df).map(tokenize, batched=True).rename_column("label", "labels")
train_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
valid_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

# Metrics to view during training
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)

    # For ROC AUC, we need probabilities rather than class predictions
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    try:
        roc_auc = roc_auc_score(labels, probs[:, 1])
    except:
        roc_auc = 0.0

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc,
    }

# Freeze layers function for ULMFiT
def freeze_until(model, block_idx):
    for name, param in model.named_parameters():
        # Always leave the classifier head trainable
        if "classifier" in name:
            param.requires_grad = True
        # Only unfreeze encoder block if block_idx ≥ 0 and matches
        elif block_idx >= 0 and f"encoder.layer.{block_idx}" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

Map:   0%|          | 0/24432 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/6736 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

## PHASE 1: Fine-tuning the Classifier Head Only

In [ ]:
# Phase 1: Head Only: encoder frozen, classifier unfrozen
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)
freeze_until(model, block_idx=-1)

# Configure the trainer to evaluate at the end of each epoch
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./ulmfit_phase1",
        eval_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        num_train_epochs=1,
        learning_rate=5e-5,
        weight_decay=0.01,
        report_to="none",
    ),
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics,
)
trainer.train()

# Save model for Phase 1 loading
model_save_path = os.path.join(save_path, "ulmfit_phase1_final")
trainer.save_model(model_save_path)
print(f"Model saved to {model_save_path}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.693200,0.686666,0.547951,0.668986,0.537843,0.884704,0.575429


Model saved to /content/drive/MyDrive/nlu/ulmfit_phase1_final


# PHASE 2: Gradual Unfreezing of Layers (24 Layers)

In [ ]:
# Phase 2: Gradual Unfreeze
num_blocks = len(model.roberta.encoder.layer)
print(f"The number of Layers is: ", num_blocks)
for i in reversed(range(num_blocks)):
    # Unfreeze the current block
    freeze_until(model, block_idx=i)

    # Set up training arguments
    phase2_args = TrainingArguments(
        output_dir=f"./ulmfit_phase2_block{i}",
        eval_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        learning_rate=5e-5 * (0.9 ** (num_blocks - i)),
        weight_decay=0.01,
        report_to="none",
        logging_strategy="epoch"
    )

    # Set up trainer
    trainer = Trainer(
        model=model,
        args=phase2_args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        compute_metrics=compute_metrics
    )

    # Train the model on this block
    trainer.train()

    # Print progress update
    print(f"Completed training with block {i} unfrozen")

# After all blocks are trained, save the final model
final_model_path = os.path.join(save_path, "ulmfit_phase2_final")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Final model saved to {final_model_path}")

The number of Layers is:  24


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.530300,0.493621,0.762619,0.782242,0.743079,0.825762,0.844457


Completed training with block 23 unfrozen


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.475000,0.475732,0.778058,0.795177,0.759487,0.834388,0.859152


Completed training with block 22 unfrozen


...

Completed training with block 2 unfrozen


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.211200,3.880069,0.828533,0.836795,0.822729,0.851351,0.888913


Completed training with block 1 unfrozen


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.119100,3.887773,0.828385,0.836953,0.821429,0.853076,0.888176


Completed training with block 0 unfrozen
Final model saved to /content/drive/MyDrive/nlu/ulmfit_phase2_final


# PHASE 3: Final Fine-Tuning of Model with SWA

### Hyperparameter Tuning for Phase 3
This will take around 6 hours.

In [ ]:
# import optuna
# from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# # Reload Phase 2 best checkpoint
# checkpoint = "./ulmfit_phase2_block0/checkpoint-3054"
# def model_init():
#     return RobertaForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# # Base TrainingArguments (will be overridden per trial)
# base_args = TrainingArguments(
#     output_dir="./ulmfit_phase3_search",
#     eval_strategy="epoch",
#     save_strategy="no",
#     load_best_model_at_end=True,
#     report_to="none",
# )

# search_trainer = Trainer(
#     model_init=model_init,
#     args=base_args,
#     train_dataset=train_ds,
#     eval_dataset=valid_ds,
#     compute_metrics=compute_metrics,
# )

# # Hyperparameter search (20 trials)
# def hp_space(trial):
#     return {
#         "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
#         "weight_decay": trial.suggest_loguniform("weight_decay", 1e-3, 1e-1),
#         "warmup_steps": trial.suggest_categorical("warmup_steps", [0, 50, 100]),
#         "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 4]),
#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8]),
#     }

# best_run = search_trainer.hyperparameter_search(
#     direction="maximize",
#     hp_space=hp_space,
#     backend="optuna",
#     n_trials=20,
# )
# print("Best hyperparameters:", best_run.hyperparameters)

# # Final full fine‑tuning with best hyperparameters
# best = best_run.hyperparameters
# phase3_args = TrainingArguments(
#     output_dir="./ulmfit_phase3_full",
#     eval_strategy="epoch",
#     save_strategy="no",
#     load_best_model_at_end=True,
#     per_device_train_batch_size=int(best["per_device_train_batch_size"]),
#     per_device_eval_batch_size=int(best["per_device_train_batch_size"]),
#     num_train_epochs=int(best["num_train_epochs"]),
#     learning_rate=best["learning_rate"],
#     weight_decay=best["weight_decay"],
#     warmup_steps=int(best["warmup_steps"]),
#     report_to="none",
# )

# trainer = Trainer(
#     model_init=model_init,
#     args=phase3_args,
#     train_dataset=train_ds,
#     eval_dataset=valid_ds,
#     compute_metrics=compute_metrics,
# )
# trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[I 2025-03-26 16:20:35,109] A new study created in memory with name: no-name-a08fc8f6-f5c2-4596-8701-f2902c464aaa
<ipython-input-20-68dc46309b3d>:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
<ipython-input-20-68dc46309b3d>:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform("weight_decay", 1e-3, 1e-1),


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.748900,0.645832,0.903207,0.908014
2,0.433500,0.944719,0.906473,0.911839
3,0.315300,1.219983,0.910778,0.915245


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.748900,0.645832,0.903207,0.908014
2,0.433500,0.944719,0.906473,0.911839
3,0.315300,1.219983,0.910778,0.915245
4,0.240300,1.365520,0.911223,0.915273


[I 2025-03-26 17:08:06,675] Trial 0 finished with value: 1.8264967264787384 and parameters: {'learning_rate': 1.5662767932032506e-06, 'weight_decay': 0.004537421892494496, 'warmup_steps': 100, 'num_train_epochs': 4, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 1.8264967264787384.
<ipython-input-20-68dc46309b3d>:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
<ipython-input-20-68dc46309b3d>:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform("weight_decay", 1e-3, 1e-1),


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.737600,0.692739,0.516330,0.681026
2,0.719900,0.692673,0.516330,0.681026
3,0.713300,0.692614,0.516330,0.681026


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.738200,0.698219,0.483670,0.000000


[I 2025-03-26 22:28:53,480] Trial 19 pruned. 
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


🏆 Best hyperparameters: {'learning_rate': 1.7631695388970417e-05, 'weight_decay': 0.0035403553886129645, 'warmup_steps': 100, 'num_train_epochs': 4, 'per_device_train_batch_size': 8}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.441800,0.342756,0.899198,0.905498
2,0.322700,0.548078,0.910629,0.914245
3,0.146700,0.544168,0.917607,0.921952
4,0.060100,0.622725,0.922506,0.925747


Final validation results: {'eval_loss': 0.34275588393211365, 'eval_accuracy': 0.8991983372921615, 'eval_f1': 0.9054975643702158, 'eval_runtime': 39.5423, 'eval_samples_per_second': 170.349, 'eval_steps_per_second': 21.294, 'epoch': 4.0}


In [ ]:
# Custom SWA Callback
class SimpleSWA(TrainerCallback):
    def __init__(self, start_step, freq):
        self.start_step = start_step
        self.freq = freq
        self.swa_model = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.swa_model = AveragedModel(kwargs["model"])

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step >= self.start_step and state.global_step % self.freq == 0:
            self.swa_model.update_parameters(kwargs["model"])

    def on_train_end(self, args, state, control, **kwargs):
        kwargs["model"].load_state_dict(self.swa_model.module.state_dict())

# Reload Phase 2 checkpoint
checkpoint = "/content/drive/MyDrive/nlu/ulmfit_phase2_final"

# Load the model
def model_init():
    config = RobertaConfig.from_pretrained(checkpoint, local_files_only=True)
    model = RobertaForSequenceClassification(config)
    ckpt_file = os.path.join(checkpoint, "model.safetensors")
    state_dict = load_file(ckpt_file)
    model.load_state_dict(state_dict)
    return model

# Phase 3 Training Arguments from Hyperparameter Tuning
phase3_args = TrainingArguments(
    output_dir="./ulmfit_phase3_swa",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to=[]
)

# Compute total steps for SWA scheduling
total_steps = len(train_ds) // phase3_args.per_device_train_batch_size * phase3_args.num_train_epochs
start_step = int(total_steps * 0.5)
swa_callback = SimpleSWA(start_step=start_step, freq=10)

trainer = Trainer(
    model_init=model_init,
    args=phase3_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics,
    callbacks=[swa_callback]
)
trainer.train()

# Final Evaluation
metrics = trainer.evaluate()
print("Final validation results with SWA:", metrics)

# Store Final Model
save_path = "/content/drive/MyDrive/nlu/swa_final_model_last/"

# Save both model weights and tokenizer
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.414500,0.338292,0.909293,0.913786,0.897201,0.930995,0.962633
2,0.251100,0.484854,0.917013,0.921145,0.904182,0.938758,0.969711


Final validation results with SWA: {'eval_loss': 0.47662946581840515, 'eval_accuracy': 0.9201306413301663, 'eval_f1': 0.9233181299885975, 'eval_precision': 0.9154889768230638, 'eval_recall': 0.9312823461759632, 'eval_roc_auc': 0.9705726356425779, 'eval_runtime': 40.075, 'eval_samples_per_second': 168.085, 'eval_steps_per_second': 21.011, 'epoch': 2.0}
Model and tokenizer saved to /content/drive/MyDrive/nlu/swa_final_model_last/


## Generate predictions.csv for Codabench

In [ ]:
# Load Model
save_path = "/content/drive/MyDrive/nlu/swa_final_model_last"
config = RobertaConfig.from_pretrained(save_path)
model = RobertaForSequenceClassification(config)
state_dict = load_file(os.path.join(save_path, "model.safetensors"))
model.load_state_dict(state_dict)
eval_trainer = Trainer(model=model)
print("Model loaded from safetensors successfully")


# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(save_path)
def tokenize(batch):
    return tokenizer(batch["premise"], batch["hypothesis"],
                     truncation=True, padding="max_length", max_length=128)

# Load the dataset and csv
dataset_df = pd.read_csv('/content/drive/MyDrive/nlu/dev.csv')
dataset_df["label"] = dataset_df["label"].astype(int)

dataset_ds = Dataset.from_pandas(dataset_df).map(tokenize, batched=True).rename_column("label", "labels")
dataset_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

# Run inference on your validation/dev dataset
pred_output = eval_trainer.predict(dataset_ds)
pred_logits = pred_output.predictions
pred_labels = np.argmax(pred_logits, axis=1)


pd.DataFrame({"prediction": pred_labels}).to_csv("predictions.csv", index=False)

print("Saved predictions.csv with", len(pred_labels), "rows.")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Model loaded from safetensors successfully


Map:   0%|          | 0/6736 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Saved predictions.csv with 6736 rows.
